In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd 'drive/My Drive/dataset/image_classification'

/content/drive/My Drive/dataset/image_classification


In [3]:
from keras.preprocessing import image
import numpy as np
import os
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [4]:
PATH = os.getcwd()
print(PATH)

/content/drive/My Drive/dataset/image_classification


In [5]:
data_dir_list = os.listdir(PATH)
data_dir_list

['dogs', 'humans', 'horses', 'cats']

In [6]:
img_data_list=[]
for dataset in data_dir_list:
  img_list=os.listdir(PATH+'/'+ dataset)
  print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
  for img in img_list:
    img_path = PATH + '/'+ dataset + '/'+ img
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    img_data_list.append(x)

Loaded the images of dataset-dogs

Loaded the images of dataset-humans

Loaded the images of dataset-horses

Loaded the images of dataset-cats



In [7]:
img_data = np.array(img_data_list)
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

(100, 1, 224, 224, 3)
(1, 100, 224, 224, 3)
(100, 224, 224, 3)


In [8]:
# Define the number of classes
num_classes = 4
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

In [9]:
labels[0:25]=0
labels[26:50]=1
labels[51:75]=2
labels[76:]=3

In [10]:
names = ['cats','dogs','horses','humans']
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

In [11]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [12]:
#Importing the necessary libraries 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D, Input
from keras.models import Model

#Building up a Sequential model
from keras.applications.resnet50 import ResNet50, preprocess_input

image_input = Input(shape=(224, 224, 3))

resnetModel = ResNet50(input_tensor = image_input, include_top=True, weights='imagenet')
resnetModel.summary()
last_layer = resnetModel.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(num_classes, activation='softmax', name='output_layer')(x)
model = Model(inputs = image_input, outputs= out)
model.summary()

102973440/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
3/3 [==============================] - 59s 16s/step - loss: 1.3050 - accuracy: 0.4187 - val_loss: 16.7870 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 50s 16s/step - loss: 0.3185 - accuracy: 0.9313 - val_loss: 243.7334 - val_accuracy: 0.2500
Epoch 3/10
3/3 [==============================] - 50s 16s/step - loss: 0.3634 - accuracy: 0.9031 - val_loss: 2294.7788 - val_accuracy: 0.3000
Epoch 4/10
3/3 [==============================] - 50s 15s/step - loss: 0.1932 - accuracy: 0.9477 - val_loss: 8924.6260 - val_accuracy: 0.3000
Epoch 5/10
3/3 [==============================] - 51s 16s/step - loss: 0.3333 - accuracy: 0.9508 - val_loss: 7229.9805 - val_accuracy: 0.3000
Epoch 6/10
3/3 [==============================] - 50s 16s/step - loss: 0.8782 - accuracy: 0.9164 - val_loss: 36483.3828 - val_accuracy: 0.3000
Epoch 7/10
3/3 [==============================] - 50s 16s/step - loss: 0.3596 - accuracy: 0.9211 - val_loss: 41840.1016 - val_accuracy: 0.3000
Epoch 8

In [ ]:
(loss, accuracy) = model.evaluate(X_test, y_test, batch_size=10)
print("loss = {:.2f}, accuracy = {:.2f}%".format(loss,accuracy * 100))

2/2 [==============================] - 4s 1s/step - loss: 153212.0938 - accuracy: 0.2500
loss = 153212.09, accuracy = 25.00%
